# Проект по Автобрее "Корпус текстов Братьев Стругацких"
## Часть 1 Создание корпуса

#### Выполнен студентом группы БКЛ211 Бахваловым Дмитрием

### Ход работы

Сначала подготовлю корпус текстов для проекта. Идея состоит в том, чтобы сделать корпус текстов Аркадия и Бориса Стругацких, но так как есть ограничение в 10000 слов и необходимо не менее 100 текстов, то мы пробежимся по их произведениям и из каждого возьмем немного текста из начала произведения. Делали это мы с помощью краулера, а обкачивали сайт strugacki.ru. Ниже идет код краулера и создание с его помощью базы данных. 

In [6]:
!pip install fake-useragent

In [18]:
from fake_useragent import UserAgent
ua = UserAgent(verify_ssl=False)
import sqlite3
from bs4 import BeautifulSoup
import re
import requests

session = requests.session()
page_number=1
url = f'https://strugacki.ru/book_1/{page_number}.html'
req = session.get(url, headers={'User-Agent': ua.random})
page = req.text
soup = BeautifulSoup(page, 'html.parser')
book_pages = soup.find('div', {'class': 'cont'}).find('p', {'class': 'text'}).find_all('a')
next_book_page=len(book_pages)+2
next_book_page

60

In [90]:
#Эта функция позволяет узнать, на какой странице начинается новая книга, т.к. нумерация страниц сплошная
def new_book_page(page_number, counter):
    url = f'https://strugacki.ru/book_{counter}/{page_number}.html'
    req = session.get(url, headers={'User-Agent': ua.random})
    page = req.text
    soup = BeautifulSoup(page, 'html.parser')
    book_pages = soup.find('div', {'class': 'cont'}).find('p', {'class': 'text'}).find_all('a')
    next_book_page=len(book_pages)+int(page_number)+1
    old_book_title = soup.find('div', {'class': 'cont'}).find('h1').text.split('>')[1].strip()
    return next_book_page, old_book_title

In [111]:
di={}
di[1]='-'
ar=['1']
for i in ar:
    try:
        di.update({new_book_page(i, len(di))[0]:new_book_page(i, len(di))[1]})
        ar.append(new_book_page(i, len(di))[0])
    except:
        Exception
#Изначально мне нужно было знать не только где начинается, но и где заканчивается текст произведения. 
#В конечном словаре это лишнее, поэтому сдвигаю значения и убираю последний ключ.
keys=list(di.keys())
val=list(di.values())
di=dict(zip(keys, val[1:len(val)+1]))

{1: 'Трудно быть богом',
 60: 'Пикник на обочине',
 98: 'Парень из преисподней',
 122: 'За миллиард лет до конца света',
 154: 'Беспокойство (Улитка на склоне-1)',
 180: 'Путь на Амальтею',
 200: 'Малыш',
 238: 'Дело об убийстве, или отель "У погибшего альпиниста"',
 286: 'Хищные вещи века',
 331: 'Стажеры',
 390: 'Полдень, XXII век (Возвращение)',
 455: 'Далекая Радуга'}

Процесс пошел, но только до страницы 487 (это я узнал из прошлой ячейки до сдвига значений словаря). Я посмотрел сайт, в чем же проблема, и там дальше нет страниц до 544. Вероятно, была какая-то книга, которую в итоге удалили. Ладно, берем теперь за точку отсчета 544 и едем дальше.

In [118]:
di_1={}
di_1[544]='-'
ar_1=[544]
for i in ar_1:
    try:
        di_1.update({new_book_page(i, len(di_1)+12)[0]:new_book_page(i, len(di_1)+12)[1]})
        ar_1.append(new_book_page(i, len(di_1)+12)[0])
    except:
        Exception
keys_1=list(di_1.keys())
val_1=list(di_1.values())
di_1=dict(zip(keys_1, val_1[1:len(val_1)+1]))
#Склеим теперь словари в один
di.update(di_1)

{1: 'Трудно быть богом',
 60: 'Пикник на обочине',
 98: 'Парень из преисподней',
 122: 'За миллиард лет до конца света',
 154: 'Беспокойство (Улитка на склоне-1)',
 180: 'Путь на Амальтею',
 200: 'Малыш',
 238: 'Дело об убийстве, или отель "У погибшего альпиниста"',
 286: 'Хищные вещи века',
 331: 'Стажеры',
 390: 'Полдень, XXII век (Возвращение)',
 455: 'Далекая Радуга',
 544: 'Дьявол среди людей',
 577: 'Дни затмения',
 593: 'Обитаемый остров',
 673: 'Попытка к бегству',
 703: 'Жук в муравейнике',
 758: 'Улитка на склоне',
 807: 'Подробности жизни Никиты Воронцова',
 817: 'Экспедиция в преисподнюю',
 879: 'Второе нашествие марсиан',
 903: 'Гадкие  лебеди',
 954: 'Волны гасят ветер',
 1003: 'Град  обреченный',
 1099: 'Понедельник начинается в субботу',
 1155: 'Сказка о тройке',
 1200: 'Отягощенные злом, или сорок лет спустя',
 1252: 'Страна багровых туч',
 1328: 'Без оружия',
 1349: 'Благоустроенная планета',
 1356: 'Хромая судьба',
 1438: 'Четвертое царство (На грани возможного)',
 1

Выкачались все произведенния с этого сайта. К сожалению, получилось их меньше 100, всего примерно 70 штук. В таком случае, возьмем из них чуть больше, чем по 100 слов из каждого.

In [124]:
for i in di.keys():
    url = f'https://strugacki.ru/book_1/{i}.html'
    req = session.get(url, headers={'User-Agent': ua.random})
    page = req.text
    soup = BeautifulSoup(page, 'html.parser')
    full = soup.find('div', {'class': 'cont'}).find('p', {'align':'justify'}).text
    

41

А теперь загрузим все в базу данных, параллельно добавляя тексты первых страниц.

In [150]:
import sqlite3
import unicodedata

conn = sqlite3.connect('my_corpus.db')
cur = conn.cursor()

cur.execute("""
CREATE TABLE IF NOT EXISTS db_corp 
(id INTEGER PRIMARY KEY AUTOINCREMENT, page_number int, title text, full_text text)
""")
conn.commit()

for i in di.keys():
    url = f'https://strugacki.ru/book_1/{i}.html'
    req = session.get(url, headers={'User-Agent': ua.random})
    page = req.text
    soup = BeautifulSoup(page, 'html.parser')
    full = soup.find('div', {'class': 'cont'}).find_all('p', {'class':'text', 'align':'justify'})
    for p in full:
        d = unicodedata.normalize('NFKD', p.text).strip()
    if d != '':
        cur.execute(
            """
            INSERT INTO db_corp 
                (page_number, title, full_text)
                VALUES (?, ?, ?)
            """, (
                i,
                di[i],
                d)
        )

conn.commit()
conn.close()

Отлично, база данных готова. Наверное, для удобства было бы отлично сделать еще одну таблицу с предложениями из каждого текста.
Наверное, для оптимизации будет хорошо, если эти предложения еще сразу разметить. 

In [74]:
import sqlite3
import unicodedata
import stanza

stz = stanza.Pipeline(lang='ru', processors='tokenize,pos,lemma')

2023-10-23 15:11:39 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


2023-10-23 15:11:40 INFO: Loading these models for language: ru (Russian):
| Processor | Package            |
----------------------------------
| tokenize  | syntagrus          |
| pos       | syntagrus_charlm   |
| lemma     | syntagrus_nocharlm |

2023-10-23 15:11:40 INFO: Using device: cpu
2023-10-23 15:11:40 INFO: Loading: tokenize
2023-10-23 15:11:40 INFO: Loading: pos
2023-10-23 15:11:41 INFO: Loading: lemma
2023-10-23 15:11:41 INFO: Done loading processors!


In [75]:
conn = sqlite3.connect('my_corpus (2).db')
cur = conn.cursor()


cur.execute("""
CREATE TABLE IF NOT EXISTS db_meta
(page_number INT, sentence TEXT, words TEXT)
""")
conn.commit()


data = """SELECT page_number, full_text FROM db_corp"""
cur.execute(data)
records = cur.fetchall()
for i in records:
    doc=stz(i[1])
    for a, snt in enumerate(doc.sentences):
        #думаю, для проекта по 25  предложений из каждой книги будет достаточно, так что на этом этапе отсеку все остальное
        if a<25:
            f=''
            for word in snt.words:
                meta='='.join([word.text, word.upos, word.lemma]) + ' '
                f+=meta
            cur.execute(
                """
                INSERT INTO db_meta
                    (page_number, sentence, words)
                    VALUES (?, ?, ?)
                """, (
                        i[0],
                        snt.text,
                        f)
            )
            conn.commit()
conn.commit()
conn.close()

## Часть 2. Создание функций поиска

In [2]:
import sqlite3
import unicodedata
import stanza

stz = stanza.Pipeline(lang='ru', processors='tokenize,pos,lemma')

2023-10-27 11:56:53 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


2023-10-27 11:56:57 INFO: Loading these models for language: ru (Russian):
| Processor | Package            |
----------------------------------
| tokenize  | syntagrus          |
| pos       | syntagrus_charlm   |
| lemma     | syntagrus_nocharlm |

2023-10-27 11:56:57 INFO: Using device: cpu
2023-10-27 11:56:57 INFO: Loading: tokenize
2023-10-27 11:56:57 INFO: Loading: pos
2023-10-27 11:57:00 INFO: Loading: lemma
2023-10-27 11:57:00 INFO: Done loading processors!


Близясь к концу кода я понял, что удобнее и быстрее всего выполнить поиск только один раз - по первому слову в инпуте. Дальше можно запомнить два слова, которые после, и остальные фукнции проверять уже на них. Из-за этого данный блок я перекраивал несколько раз, так что комментарии немного перепутались, но я постарался восстановить все как было

In [3]:
def exact_word(some_word):
    conn = sqlite3.connect('my_corpus (2).db')
    cur = conn.cursor()
    data = """
    SELECT db_meta.page_number, sentence, words, title FROM db_meta
    JOIN db_corp ON db_meta.page_number = db_corp.page_number
    """
    seen=[]
    cur.execute(data)
    textts=cur.fetchall()
    for row in textts:
        lst_rows=row[2].split()
        for n_l in range(len(lst_rows)):
            wrd=lst_rows[n_l].split('=')
            if wrd[0] == some_word[1:-1]:
                if n_l+2 < len(lst_rows):
                    seen.append([row[1], row[3], lst_rows[n_l+1], lst_rows[n_l+2]])
                elif n_l+1 < len(lst_rows):
                    seen.append([row[1], row[3], lst_rows[n_l+1], 'null'])
                else:
                    seen.append([row[1], row[3], 'null', 'null'])
    conn.close()
    return(seen)

In [4]:
def lemma_word(some_word):
    conn = sqlite3.connect('my_corpus (2).db')
    cur = conn.cursor()
    data = """
    SELECT db_meta.page_number, sentence, words, title FROM db_meta
    JOIN db_corp ON db_meta.page_number = db_corp.page_number
    """
    seen=[]
    cur.execute(data)
    textts=cur.fetchall()
    doc=stz(some_word)
    for sent in doc.sentences:
        for word in sent.words:
            some_word_lem=word.lemma
    for row in textts:
        k=0
        lst_rows=row[2].split()
        for n_l in range(len(lst_rows)):
            k+=1
            wrd=lst_rows[n_l].split('=')
            if wrd[2] == some_word_lem:
                if n_l+2 < len(lst_rows):
                    seen.append([row[1], row[3], lst_rows[n_l+1], lst_rows[n_l+2]])
                elif n_l+1 < len(lst_rows):
                    seen.append([row[1], row[3], lst_rows[n_l+1], 'null'])
                else:
                    seen.append([row[1], row[3], 'null', 'null'])
    conn.close()
    return(seen)



In [5]:
def word_and_pos(some_word):
    some_word=some_word.split('+')
    conn = sqlite3.connect('my_corpus (2).db')
    cur = conn.cursor()
    data = """
    SELECT db_meta.page_number, sentence, words, title FROM db_meta
    JOIN db_corp ON db_meta.page_number = db_corp.page_number
    """
    seen=[]
    cur.execute(data)
    textts=cur.fetchall()
    for row in textts:
        lst_rows=row[2].split()
        for n_l in range(len(lst_rows)):
            wrd=lst_rows[n_l].split('=')
            if wrd[0] == some_word[0] and wrd[1] == some_word[1]:
                if n_l+2 < len(lst_rows):
                    seen.append([row[1], row[3], lst_rows[n_l+1], lst_rows[n_l+2]])
                elif n_l+1 < len(lst_rows):
                    seen.append([row[1], row[3], lst_rows[n_l+1], 'null'])
                else:
                    seen.append([row[1], row[3], 'null', 'null'])
    conn.close()
    return(seen)

Вид запроса слово+POS у нас ищет слова именно в этой форме.  

In [6]:
def pos_word(pos):
    conn = sqlite3.connect('my_corpus (2).db')
    cur = conn.cursor()
    data = """
    SELECT db_meta.page_number, sentence, words, title FROM db_meta
    JOIN db_corp ON db_meta.page_number = db_corp.page_number
    """
    seen=[]
    cur.execute(data)
    textts=cur.fetchall()
    for row in textts:
        lst_rows=row[2].split()
        for n_l in range(len(lst_rows)):
            wrd=lst_rows[n_l].split('=')
            if wrd[1] == pos:
                if n_l+2 < len(lst_rows):
                    seen.append([row[1], row[3], lst_rows[n_l+1], lst_rows[n_l+2]])
                elif n_l+1 < len(lst_rows):
                    seen.append([row[1], row[3], lst_rows[n_l+1], 'null'])
                else:
                    seen.append([row[1], row[3], 'null', 'null'])
    conn.close()
    return(seen)

Вот функция для записи одного слова:

In [7]:
#на вход str
def for_one_word(word_s):
    frst_wrd=[]
    if word_s[0]=='"':
        for i in exact_word(word_s):
            frst_wrd.append([i[0], i[1], i[2], i[3]])
    elif word_s[0] in 'QWERTYUIOPASDFGHJKLZXCVBNM':
        for i in pos_word(word_s):
            frst_wrd.append([i[0], i[1], i[2], i[3]])
    elif len(word_s.split('+'))==2:
        for i in word_and_pos(word_s):
            frst_wrd.append([i[0], i[1], i[2], i[3]])
    else:
        for i in lemma_word(word_s):
            frst_wrd.append([i[0], i[1], i[2], i[3]])
    return(frst_wrd)

In [8]:
#на вход получаем список из двух слов, по которым поиск
def for_two_words(word_s):
    two_wrds=[]
    for o in for_one_word(word_s[0]):
        if word_s[1][0]=='"':
            if o[2].split('=')[0] == word_s[1][1:-1]:
                two_wrds.append(o)
        elif word_s[1][0] in 'QWERTYUIOPASDFGHJKLZXCVBNM':
            if len(o[2].split('=')) != 1:
                if o[2].split('=')[1] == word_s[1]:
                    two_wrds.append(o)
        elif len(word_s[1].split('+'))==2:
            if len(o[2].split('=')) != 1:
                if o[2].split('=')[1] == word_s[1].split('+')[1] and o[2].split('=')[0] == word_s[1].split('+')[0]:
                    two_wrds.append(o)
        else:
            doc=stz(word_s[1])
            for sent in doc.sentences:
                for word in sent.words:
                    word_lem=word.lemma
            if len(o[2].split('=')) != 1:
                if o[2].split('=')[2] == word_lem:
                    two_wrds.append(o)
    return(two_wrds)

In [9]:
def for_three_words(word_s):
    three_wrds=[]
    for o in for_two_words(word_s[:2]):
        if word_s[2][0]=='"':
            if o[3].split('=')[0] == word_s[2][1:-1]:
                three_wrds.append(o)
        elif word_s[2][0] in 'QWERTYUIOPASDFGHJKLZXCVBNM':
            if len(o[3].split('=')) != 1:
                if o[3].split('=')[1] == word_s[2]:
                    three_wrds.append(o)
        elif len(word_s[2].split('+'))==2:
            if len(o[3].split('=')) != 1:
                if o[3].split('=')[1] == word_s[2].split('+')[1] and o[3].split('=')[0] == word_s[2].split('+')[0]:
                    three_wrds.append(o)
        else:
            doc=stz(word_s[2])
            for sent in doc.sentences:
                for word in sent.words:
                    word_lem=word.lemma
            if len(o[3].split('=')) != 1:
                if o[3].split('=')[2] == word_lem:
                    three_wrds.append(o)
    return(three_wrds)

In [10]:
def search(inp):
    d=[]
    if len(inp) == 1:
        for o in for_one_word(inp[0]):
            d.append(o)
    elif len(inp) == 2:
        for o in for_two_words(inp[:2]):
            d.append(o)
    elif len(inp) == 3:
        for o in for_three_words(inp[:3]):
            d.append(o)
    return d

In [22]:
inp=input()
inp=inp.split()
for o in search(inp):
    print( o[0], f'["{o[1]}", Аркадий и Борис Стругацкие. Ресурс: https://strugacki.ru/ ]')    

ADP NOUN VERB
— Да, с открытиями происходят иногда удивительные вещи. ["Пикник на обочине", Аркадий и Борис Стругацкие. Ресурс: https://strugacki.ru/ ]
Дырки на глобусе лягут на некую плавную кривую. ["Пикник на обочине", Аркадий и Борис Стругацкие. Ресурс: https://strugacki.ru/ ]
На площади около фактории дымила полевая кухня, снятая с колес. ["Парень из преисподней", Аркадий и Борис Стругацкие. Ресурс: https://strugacki.ru/ ]
На кухонном столике среди хлебных крошек красовался натюрморт из сковородки с засохшими остатками яичницы, недопитого стакана чая и обкусанной горбушки со следами оплавившегося масла. ["За миллиард лет до конца света", Аркадий и Борис Стругацкие. Ресурс: https://strugacki.ru/ ]
От звездного блеска на равнине лежат неясные отсветы, а скалистый хребет кажется глубокой черной тенью на звездном небе. ["Путь на Амальтею", Аркадий и Борис Стругацкие. Ресурс: https://strugacki.ru/ ]
Бывает, что низко над хребтом висит пятнистый серп Ганимеда, или серебряный диск 